# Comparison Between TreeValue and Tianshou Batch

In this section, we will take a look at the feature and performance of the [Tianshou Batch](https://github.com/thu-ml/tianshou) library, which is developed by Tsinghua Machine Learning Group.

Before starting the comparison, let us define some thing.

In [1]:
import torch 

_TREE_DATA_1 = {'a': 1, 'b': 2, 'x': {'c': 3, 'd': 4}}
_TREE_DATA_2 = {
    'a': torch.randn(2, 3), 
    'x': {
        'c': torch.randn(3, 4)
    },
}
_TREE_DATA_3 = {
    'obs': torch.randn(4, 84, 84),
    'action': torch.randint(0, 6, size=(1,)),
    'reward': torch.rand(1),
}

/opt/hostedtoolcache/Python/3.8.16/x64/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Read and Write Operation

Reading and writing are the two most common operations in the tree data structure based on the data model (TreeValue and Tianshou Batch both belong to this type), so this section will compare the reading and writing performance of these two libraries.

### TreeValue's Get and Set

In [2]:
from treevalue import FastTreeValue

t = FastTreeValue(_TREE_DATA_2)

In [3]:
t

<FastTreeValue 0x7f76b8c4eeb0>
├── 'a' --> tensor([[ 1.5521, -0.4073,  0.6176],
│                   [-0.3600, -0.4997, -1.4075]])
└── 'x' --> <FastTreeValue 0x7f76b8c4e100>
    └── 'c' --> tensor([[ 1.1370,  0.6563,  0.1383, -0.5934],
                        [-2.1173, -1.4629,  0.6929, -0.9648],
                        [ 1.1643, -0.0296, -0.1839,  0.0326]])

In [4]:
t.a

tensor([[ 1.5521, -0.4073,  0.6176],
        [-0.3600, -0.4997, -1.4075]])

In [5]:
%timeit t.a

72.1 ns ± 0.421 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [6]:
new_value = torch.randn(2, 3)
t.a = new_value

t

<FastTreeValue 0x7f76b8c4eeb0>
├── 'a' --> tensor([[-0.2235, -1.2315, -1.6898],
│                   [ 0.1621,  0.0774, -0.8981]])
└── 'x' --> <FastTreeValue 0x7f76b8c4e100>
    └── 'c' --> tensor([[ 1.1370,  0.6563,  0.1383, -0.5934],
                        [-2.1173, -1.4629,  0.6929, -0.9648],
                        [ 1.1643, -0.0296, -0.1839,  0.0326]])

In [7]:
%timeit t.a = new_value

74.2 ns ± 1.37 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


### Tianshou Batch's Get and Set

In [8]:
from tianshou.data import Batch

b = Batch(**_TREE_DATA_2)

In [9]:
b

Batch(
    a: tensor([[ 1.5521, -0.4073,  0.6176],
               [-0.3600, -0.4997, -1.4075]]),
    x: Batch(
           c: tensor([[ 1.1370,  0.6563,  0.1383, -0.5934],
                      [-2.1173, -1.4629,  0.6929, -0.9648],
                      [ 1.1643, -0.0296, -0.1839,  0.0326]]),
       ),
)

In [10]:
b.a

tensor([[ 1.5521, -0.4073,  0.6176],
        [-0.3600, -0.4997, -1.4075]])

In [11]:
%timeit b.a

75.2 ns ± 1.47 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [12]:
new_value = torch.randn(2, 3)
b.a = new_value

b

Batch(
    a: tensor([[ 1.5253,  0.2708, -0.7370],
               [-0.5360, -0.3040, -0.9124]]),
    x: Batch(
           c: tensor([[ 1.1370,  0.6563,  0.1383, -0.5934],
                      [-2.1173, -1.4629,  0.6929, -0.9648],
                      [ 1.1643, -0.0296, -0.1839,  0.0326]]),
       ),
)

In [13]:
%timeit b.a = new_value

616 ns ± 7.14 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


## Initialization

### TreeValue's Initialization

In [14]:
%timeit FastTreeValue(_TREE_DATA_1)

990 ns ± 15.6 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


### Tianshou Batch's Initialization

In [15]:
%timeit Batch(**_TREE_DATA_1)

13.2 µs ± 195 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


## Deep Copy Operation

In [16]:
import copy

### Deep Copy of TreeValue

In [17]:
t3 = FastTreeValue(_TREE_DATA_3)
%timeit copy.deepcopy(t3)

294 µs ± 12 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


### Deep Copy of Tianshou Batch

In [18]:
b3 = Batch(**_TREE_DATA_3)
%timeit copy.deepcopy(b3)

284 µs ± 13.9 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


## Stack, Concat and Split Operation

### Performance of TreeValue

In [19]:
trees = [FastTreeValue(_TREE_DATA_2) for _ in range(8)]

In [20]:
t_stack = FastTreeValue.func(subside=True)(torch.stack)

t_stack(trees)

<FastTreeValue 0x7f76b8c4e9d0>
├── 'a' --> tensor([[[ 1.5521, -0.4073,  0.6176],
│                    [-0.3600, -0.4997, -1.4075]],
│           
│                   [[ 1.5521, -0.4073,  0.6176],
│                    [-0.3600, -0.4997, -1.4075]],
│           
│                   [[ 1.5521, -0.4073,  0.6176],
│                    [-0.3600, -0.4997, -1.4075]],
│           
│                   [[ 1.5521, -0.4073,  0.6176],
│                    [-0.3600, -0.4997, -1.4075]],
│           
│                   [[ 1.5521, -0.4073,  0.6176],
│                    [-0.3600, -0.4997, -1.4075]],
│           
│                   [[ 1.5521, -0.4073,  0.6176],
│                    [-0.3600, -0.4997, -1.4075]],
│           
│                   [[ 1.5521, -0.4073,  0.6176],
│                    [-0.3600, -0.4997, -1.4075]],
│           
│                   [[ 1.5521, -0.4073,  0.6176],
│                    [-0.3600, -0.4997, -1.4075]]])
└── 'x' --> <FastTreeValue 0x7f76b8c4efd0>
    └── 'c' --> tensor([[[

In [21]:
%timeit t_stack(trees)

47.4 µs ± 1.04 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [22]:
t_cat = FastTreeValue.func(subside=True)(torch.cat)

t_cat(trees)

<FastTreeValue 0x7f760be80f40>
├── 'a' --> tensor([[ 1.5521, -0.4073,  0.6176],
│                   [-0.3600, -0.4997, -1.4075],
│                   [ 1.5521, -0.4073,  0.6176],
│                   [-0.3600, -0.4997, -1.4075],
│                   [ 1.5521, -0.4073,  0.6176],
│                   [-0.3600, -0.4997, -1.4075],
│                   [ 1.5521, -0.4073,  0.6176],
│                   [-0.3600, -0.4997, -1.4075],
│                   [ 1.5521, -0.4073,  0.6176],
│                   [-0.3600, -0.4997, -1.4075],
│                   [ 1.5521, -0.4073,  0.6176],
│                   [-0.3600, -0.4997, -1.4075],
│                   [ 1.5521, -0.4073,  0.6176],
│                   [-0.3600, -0.4997, -1.4075],
│                   [ 1.5521, -0.4073,  0.6176],
│                   [-0.3600, -0.4997, -1.4075]])
└── 'x' --> <FastTreeValue 0x7f760bf649d0>
    └── 'c' --> tensor([[ 1.1370,  0.6563,  0.1383, -0.5934],
                        [-2.1173, -1.4629,  0.6929, -0.9648],
                 

In [23]:
%timeit t_cat(trees)

46.4 µs ± 3.76 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [24]:
t_split = FastTreeValue.func(rise=True)(torch.split)
tree = FastTreeValue({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1),
})

%timeit t_split(tree, 1)

78 µs ± 1.98 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Performance of Tianshou Batch

In [25]:
batches = [Batch(**_TREE_DATA_2) for _ in range(8)]

Batch.stack(batches)

Batch(
    a: tensor([[[ 1.5521, -0.4073,  0.6176],
                [-0.3600, -0.4997, -1.4075]],
       
               [[ 1.5521, -0.4073,  0.6176],
                [-0.3600, -0.4997, -1.4075]],
       
               [[ 1.5521, -0.4073,  0.6176],
                [-0.3600, -0.4997, -1.4075]],
       
               [[ 1.5521, -0.4073,  0.6176],
                [-0.3600, -0.4997, -1.4075]],
       
               [[ 1.5521, -0.4073,  0.6176],
                [-0.3600, -0.4997, -1.4075]],
       
               [[ 1.5521, -0.4073,  0.6176],
                [-0.3600, -0.4997, -1.4075]],
       
               [[ 1.5521, -0.4073,  0.6176],
                [-0.3600, -0.4997, -1.4075]],
       
               [[ 1.5521, -0.4073,  0.6176],
                [-0.3600, -0.4997, -1.4075]]]),
    x: Batch(
           c: tensor([[[ 1.1370,  0.6563,  0.1383, -0.5934],
                       [-2.1173, -1.4629,  0.6929, -0.9648],
                       [ 1.1643, -0.0296, -0.1839,  0.0326]],
         

In [26]:
%timeit Batch.stack(batches)

106 µs ± 4.45 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [27]:
Batch.cat(batches)

Batch(
    a: tensor([[ 1.5521, -0.4073,  0.6176],
               [-0.3600, -0.4997, -1.4075],
               [ 1.5521, -0.4073,  0.6176],
               [-0.3600, -0.4997, -1.4075],
               [ 1.5521, -0.4073,  0.6176],
               [-0.3600, -0.4997, -1.4075],
               [ 1.5521, -0.4073,  0.6176],
               [-0.3600, -0.4997, -1.4075],
               [ 1.5521, -0.4073,  0.6176],
               [-0.3600, -0.4997, -1.4075],
               [ 1.5521, -0.4073,  0.6176],
               [-0.3600, -0.4997, -1.4075],
               [ 1.5521, -0.4073,  0.6176],
               [-0.3600, -0.4997, -1.4075],
               [ 1.5521, -0.4073,  0.6176],
               [-0.3600, -0.4997, -1.4075]]),
    x: Batch(
           c: tensor([[ 1.1370,  0.6563,  0.1383, -0.5934],
                      [-2.1173, -1.4629,  0.6929, -0.9648],
                      [ 1.1643, -0.0296, -0.1839,  0.0326],
                      [ 1.1370,  0.6563,  0.1383, -0.5934],
                      [-2.1173, -

In [28]:
%timeit Batch.cat(batches)

188 µs ± 4.68 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [29]:
batch = Batch({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1)}
)

%timeit list(Batch.split(batch, 1, shuffle=False, merge_last=True))

593 µs ± 48 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)
